In [239]:
import h5py
import sys
import os
import glob
import csv
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from hdfs import InsecureClient

Go through the base path and find all the .h5 files that are present there then save them to a csv file in order to load them later into spark

TODO: this actually needs to be reworked to create a table based on the hdfs file directory instead of local, however it doesn't matter as long as we don't change the files just run the code and pretend it does the right thing for now

In [240]:
basedir = '../MillionSongSubset/data'
os.listdir(basedir)

['A', 'B', 'files.csv']

In [241]:
ext='.h5'
all_files = []
for root, dirs, files in os.walk(basedir):
    files = glob.glob(os.path.join(root,'*'+ext))
    all_files.append(files)

flat_list = [item for sublist in all_files for item in sublist]

In [242]:
file = '/files.csv'
with open(basedir + file, 'w', newline='') as myfile:
    for line in flat_list:
        myfile.write(line)
        myfile.write('\n')

Set up the spark application

In [243]:
# calculate the number of partitions we want for the rdds
num_nodes = 3
num_rep = 2 * num_nodes

conf = (SparkConf()
   .setMaster("spark://192.168.2.110:7077")
   .setAppName("Group14")
   .set("spark.executor.cores", 2)
   .set("spark.pytspark.python","python3"))

sc = SparkContext(conf = conf)

In [244]:
sqlContext = SQLContext(sc)

code to convert filenames to actual file content

In [254]:
# good methods to explore the h5 inread: .keys(), .attrs.items() see documentation of h5py
import tables


def get_title(file, idx=0):
    return file['metadata']['songs']['title'][idx].decode("utf-8")

def get_artist_name(file, idx=0):
    return file['metadata']['songs']['artist_name'][idx].decode("utf-8")

# todo: implement functions to get other attributes here!!

In [255]:
# import hdf5_getters
import tables
import requests
import io

# right now only mapping for the title needs to be changed to extract all the other features
def f(x):
    # set up the url where the data is stored (yes it's ugly)
    base_hdfs = '/LDSA/data'
    url = 'http://192.168.2.110:50070/webhdfs/v1'
    op = '?op=OPEN'
    full_path = url + base_hdfs + x[2:] + op
    # read in song file from hdfs api and then extract the wanted attributes
    with h5py.File(io.BytesIO(requests.get(url = full_path).content), 'r') as f:
        song = []
        song.append(get_title(f))
        song.append(get_artist_name(f))
    return song

In [256]:
# todo: set the number of partitions here corectly as calculated above in order to maintain speed
file_paths = sc.textFile(basedir + file).map(lambda x: f(x))

convert the rdd (api version 1) to a dataframe (api version 2)

In [257]:
# todo: add the needed attributes here that we need..
df = sqlContext.createDataFrame(file_paths, schema=['title', 'artist_name'])

In [258]:
df.printSchema()

root
 |-- title: string (nullable = true)
 |-- artist_name: string (nullable = true)



start with the data processing

In [ ]:
df.show()

In [238]:
sc.stop()